In [5]:
import ollama
import ast
import json
from Levenshtein import ratio
from pprint import pprint

In [ ]:
prefix_path = '../Evaluation/Helper/plan_extraction_helper_llm.txt'

travelPlan_file_path = "../Outputs/No Limitation/6.txt"
with open(travelPlan_file_path, 'r') as f:
        plan_text = f.read()
        
output = get_helper_response(prefix_path,plan_text)
#just to make sure we grab the actuall json place, we will start with the first [ and end with the last ]
output = output[output.find('['):output.rfind(']')+1]
output_json = json.loads(output)

with open('../Evaluation/Outputs/No Limitation 6 evaluation.json', 'w') as f:
    json.dump(output_json, f)

In [12]:
def get_helper_response(helper_prefix_path, plan_text):
    with open(helper_prefix_path, 'r') as f:
        prefix = f.read()
    prompt = prefix + plan_text
    response = ollama.generate(model='llama3.1', prompt=prompt, options={'num_ctx': 10000})
    return response['response']

def find_mentioned_business(travelPlan_file_path, allBusiness_file_path):
    #load travel plan
    with open(travelPlan_file_path, 'r') as f:
        plan_text = f.read()

    #load all business
    all_business = []
    with open(allBusiness_file_path, 'r') as file:
        for line in file:
            json_obj = json.loads(line.strip())
            all_business.append(json_obj)

    #get mentioned business from the raw travel plan
    metioned_business_array = ast.literal_eval(get_helper_response(plan_text))
    

    #find the actual business from the database
    retrieved_mentioned_business = []
    score_list = []
    for metioned_business in metioned_business_array:
        scores = []
        for business in all_business:
            similarity_score = ratio(business['name'].lower(), metioned_business.lower())
            scores.append(similarity_score)
        index = scores.index(max(scores))
        score_list.append(max(scores))
        retrieved_mentioned_business.append(all_business[index])

    return retrieved_mentioned_business,score_list,metioned_business_array
    

In [ ]:
travelPlan_file_path = "../Outputs/Preference/Theme_History_1.txt"
allBusiness_file_path = "../Datasets/all_pruned.jsonl"
biz,scores,original = find_mentioned_business(travelPlan_file_path, allBusiness_file_path)
for i in range(len(biz)):
    print(biz[i]['name'] , scores[i],original[i])